In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.__version__

'2.3.0'

In [6]:
mnist=tf.keras.datasets.mnist

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

In [8]:
x_train.min(), x_train.max()

(0.0, 1.0)

In [9]:
# 차원을 1 늘려줍니다
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

In [10]:
x_train.shape, x_test.shape

(TensorShape([60000, 28, 28, 1]), TensorShape([10000, 28, 28, 1]))

# 데이터셋 만들기

In [11]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 모델 만들기

In [12]:
# layer 정의
input_ = tf.keras.layers.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(input_)
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output_ = tf.keras.layers.Dense(10, activation='softmax')(x)

# model을 정의
model = tf.keras.models.Model(input_, output_)

In [13]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4718720   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 4,738,826
Trainable params: 4,738,826
Non-trainable params: 0
____________________________________________

# 손실함수 정의

In [15]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimizer 정의

In [17]:
optimizer = tf.keras.optimizers.Adam()

# 성과척도 정의

In [19]:
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

In [20]:
test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# 훈련/테스트 스텝 함수 정의

## GradientTape(그래디언트 테이프)를 사용하자

In [26]:
@tf.function
def train_step(images, labels):
    # 미분을 위해 GradientTape을 적용.
    with tf.GradientTape() as tape:
        # 1. 예측 (prediction)
        #  to do
        # 2. Loss 계산
        #  to do
    
    # 3. 그라디언트(gradients) 계산
    # to do
    
    # 4. 오차 역전파(Backpropagation) - 가중치와 편향 업데이트
    # to do
    
    # 5. loss와 accuracy를 업데이트
    train_loss(loss)
    train_acc(labels, predictions)

In [27]:
@tf.function
def test_step(images, labels):
    # 1. 예측 (prediction)
    predictions = model(images)
    # 2. Loss 계산
    loss = loss_function(labels, predictions)
    
    # Test셋에 대해서는 gradient를 계산 및 backpropagation 하지 않는다.
    
    # 3.loss와 accuracy를 업데이트
    test_loss(loss)
    test_acc(labels, predictions)

In [25]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {:.5f}, 정확도: {:.2f}%, 테스트 손실: {:.5f}, 테스트 정확도: {:.2f}%'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_acc.result()*100,
                           test_loss.result(),
                           test_acc.result()*100))

에포크: 1, 손실: 0.10281, 정확도: 96.86%, 테스트 손실: 0.05098, 테스트 정확도: 98.39%
에포크: 2, 손실: 0.06782, 정확도: 97.92%, 테스트 손실: 0.05075, 테스트 정확도: 98.43%
에포크: 3, 손실: 0.05156, 정확도: 98.41%, 테스트 손실: 0.05106, 테스트 정확도: 98.45%
에포크: 4, 손실: 0.04153, 정확도: 98.72%, 테스트 손실: 0.05053, 테스트 정확도: 98.51%
에포크: 5, 손실: 0.03514, 정확도: 98.91%, 테스트 손실: 0.05011, 테스트 정확도: 98.55%
